In [ ]:
import os
import json
import requests
from time import sleep, time
from dotenv import load_dotenv
load_dotenv()
USERNAME = os.getenv("MERCK_USERNAME")
PASSWORD = os.getenv("MERCK_PASSWORD")

if not USERNAME or not PASSWORD:
   raise RuntimeError("Please set MERCK_USERNAME and MERCK_PASSWORD in your .env file")

BASE_URL = "https://biophi-dev.merck.com/api"

In [ ]:
def run_biophi(heavy_chain: str, light_chain: str = "", name: str = "seq_1", pause: int = 30) -> dict:
  
   """
   Submit a single-sequence job to BioPhi, poll until complete,
   and return the full JSON result.
   """
  
   r = requests.post(f'{BASE_URL}/login', auth=(USERNAME, PASSWORD), verify=False)
   r.raise_for_status()
  
   headers = {'Cookie': r.headers['set-cookie'].split(';')[0]}
  
   print('\nSubmitting Job to BioPhi...\n---------------------------')
  
   input_json = {
       "sequences": [
           {
               "name": name,
               "HC": heavy_chain,
               "LC": light_chain
           }
       ]
   }


   print(f"Submitting payload: {input_json}")
  
   submission_response = requests.post(
       f'{BASE_URL}/reports',
       json=input_json,
       headers=headers,
       verify=False
   )
   submission_response.raise_for_status()
   submission = submission_response.json()
  
   print('\nData URLs:\n----------')
   print(submission)
  
   print('\nRunning the report...\n---------------------')
   summary_url = submission['url_summary']
  
   running = True
   start_time = time()
  
   while running:
       sleep(pause)
       summary_report = requests.get(
           summary_url,
           auth=(USERNAME, PASSWORD),
           headers=headers,
           verify=False
       ).json()
      
       try:
           running = summary_report['in_progress']
       except:
           print(summary_report)
      
       current_time = time()
       if running:
           print(f'Report still running after {round(current_time-start_time, 2)} seconds...')
  
   end_time = time()
   print(f'\nReport done after {round(end_time-start_time, 2)} seconds.')
  
   print('\nFetching results...\n---------------------------')
  
   result = {}
   for key, url in submission.items():
       if key == 'report_id':
           result[key] = url
       else:
           data_json = requests.get(
               url,
               auth=(USERNAME, PASSWORD),
               headers=headers,
               verify=False
           ).json()
           result[key] = data_json
           print(f'{key} data retrieved')
  
   print('\nDone fetching data.\n')
   return result


In [ ]:
heavy_chain_sequence = "EVQLVESGGGLVQPGGSLRLSCAASGFTFKSYAMDWVRQAPGKQREWVSAISQSGYSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCARRYRVRWGFDVWGQGTLVTVSS"
light_chain = ""


result = run_biophi(heavy_chain_sequence, light_chain, name="just_heavy")
print(json.dumps(result, indent=4))
os.makedirs('../data/biophi', exist_ok=True)
with open('../data/biophi/just_heavy.json', 'w') as f:
   json.dump(result, f, indent=2)